# ELEC 475 Lab 4 - CLIP Visualization (FINAL FIX)

**FIXES:**
1. ✅ Auto-fixes text_encoder.text_model key mismatch
2. ✅ **NEW: Properly loads text_projection layer**

This should fix the low similarity (0.10) issue!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q transformers torch torchvision tqdm pillow matplotlib

In [ ]:
import os
if not os.path.exists('475_ML-CV_Labs'):
    !git clone https://github.com/Jcub05/475_ML-CV_Labs.git
os.chdir('/content/475_ML-CV_Labs/Lab4')
print(f"✓ Directory: {os.getcwd()}")

In [ ]:
import urllib.request
import zipfile
from pathlib import Path

data_dir = Path('/content/coco_data')
data_dir.mkdir(exist_ok=True)
val_dir = data_dir / 'val2014'

if not val_dir.exists() or len(list(val_dir.glob('*.jpg'))) == 0:
    val_url = 'http://images.cocodataset.org/zips/val2014.zip'
    val_zip = data_dir / 'val2014.zip'
    urllib.request.urlretrieve(val_url, val_zip)
    with zipfile.ZipFile(val_zip, 'r') as z:
        z.extractall(data_dir)
    val_zip.unlink()

print(f"✓ Images: {len(list(val_dir.glob('*.jpg')))}")

In [ ]:
# CONFIG
MODEL_TYPE = 'batchnorm'
model_checkpoint_path = '/content/drive/MyDrive/elec475_lab4/models/best_model_batch_norm.pth'

print(f"Model: {MODEL_TYPE}")
print(f"Path: {model_checkpoint_path}")

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from torchvision import transforms
from PIL import Image

from model import CLIPFineTuneModel
from model_modified import CLIPImageEncoderModified, CLIPFineTuneModelModified
from visualize import visualize_text_to_image_retrieval, zero_shot_classification, create_retrieval_grid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
output_dir = Path('/content/Visualizations')
output_dir.mkdir(exist_ok=True)

print(f"Device: {device}")

In [ ]:
def get_clip_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.48145466, 0.4578275, 0.40821073),
            std=(0.26862954, 0.26130258, 0.27577711)
        )
    ])

def load_model_with_architecture(model_path, model_type, device):
    print(f"\nLoading {model_type} model...")
    
    MODEL_CONFIGS = {
        'base': {'use_batchnorm': False, 'use_dropout': False, 'deeper_projection': False},
        'batchnorm': {'use_batchnorm': True, 'use_dropout': False, 'deeper_projection': False},
        'dropout': {'use_batchnorm': False, 'use_dropout': True, 'dropout_rate': 0.1, 'deeper_projection': False},
    }
    
    # Create model
    if model_type == 'base':
        model = CLIPFineTuneModel(
            embed_dim=512,
            pretrained_resnet=True,
            clip_model_name="openai/clip-vit-base-patch32",
            freeze_text_encoder=True
        ).to(device)
    else:
        config = MODEL_CONFIGS[model_type]
        image_encoder = CLIPImageEncoderModified(embed_dim=512, **config)
        
        # ✅ FIX: Load FULL CLIP model to get text_projection!
        clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        
        model = CLIPFineTuneModelModified(
            image_encoder=image_encoder,
            text_encoder=clip_model.text_model,
            tokenizer=None
        ).to(device)
        
        # ✅ NEW: Add text_projection as an attribute!
        model.text_projection = clip_model.text_projection.to(device)
    
    # Load checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    state_dict = checkpoint.get('model_state_dict', checkpoint)
    
    # Fix text_encoder keys
    fixed_state_dict = {}
    num_fixed = 0
    for key, value in state_dict.items():
        if key.startswith('text_encoder.text_model.'):
            new_key = key.replace('text_encoder.text_model.', 'text_encoder.')
            fixed_state_dict[new_key] = value
            num_fixed += 1
        else:
            fixed_state_dict[key] = value
    
    if num_fixed > 0:
        print(f"✓ Fixed {num_fixed} text_encoder keys")
    
    # Load weights (strict=False to ignore text_projection if not in checkpoint)
    missing, unexpected = model.load_state_dict(fixed_state_dict, strict=False)
    
    if 'text_projection' in str(missing):
        print("✓ text_projection loaded from pretrained CLIP (not in checkpoint)")
    
    model.eval()
    print(f"✓ Model ready\n")
    return model

def precompute_image_embeddings(model, image_paths, transform, device, batch_size=32):
    print(f"Precomputing {len(image_paths)} image embeddings...")
    all_embeds = []
    model.eval()
    
    with torch.no_grad():
        for i in range(0, len(image_paths), batch_size):
            batch_paths = image_paths[i:i+batch_size]
            images = [transform(Image.open(p).convert('RGB')) for p in batch_paths]
            images = torch.stack(images).to(device)
            embeds = model.encode_image(images).cpu()
            all_embeds.append(embeds)
            
            if (i // batch_size + 1) % 10 == 0:
                print(f"  {i+len(batch_paths)}/{len(image_paths)}")
    
    all_embeds = torch.cat(all_embeds, dim=0)
    print(f"✓ Shape: {all_embeds.shape}")
    return all_embeds

class ModifiedModelWrapper:
    def __init__(self, model, processor):
        self.model = model
        self.processor = processor
        
    def eval(self):
        self.model.eval()
        return self
    
    def encode_text(self, input_ids, attention_mask):
        """✅ FIXED: Now uses text_projection!"""
        with torch.no_grad():
            if hasattr(self.model, 'text_encoder'):
                outputs = self.model.text_encoder(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )
                
                # ✅ FIX: Apply text_projection!
                if hasattr(self.model, 'text_projection'):
                    text_embeds = self.model.text_projection(outputs.pooler_output)
                else:
                    text_embeds = outputs.pooler_output
                
                import torch.nn.functional as F
                return F.normalize(text_embeds, p=2, dim=-1)
            else:
                return self.model.encode_text(input_ids, attention_mask)
    
    def encode_image(self, images):
        return self.model.encode_image(images)

def generate_visualizations(model, model_name, image_paths, image_embeds, processor, transform, device, output_dir):
    print(f"\nGenerating visualizations for: {model_name}")
    model_output_dir = output_dir / model_name
    model_output_dir.mkdir(parents=True, exist_ok=True)
    
    wrapped_model = ModifiedModelWrapper(model, processor) if isinstance(model, CLIPFineTuneModelModified) else model
    
    text_queries = ["sport", "a dog playing", "a person eating", "a beautiful sunset", "a cat on a couch"]
    
    for query in text_queries:
        print(f"  '{query}'")
        visualize_text_to_image_retrieval(
            query_text=query,
            model=wrapped_model,
            image_paths=image_paths,
            image_embeds=image_embeds,
            clip_processor=processor,
            device=device,
            top_k=5,
            save_path=model_output_dir / f"text2img_{query.replace(' ', '_')}.png"
        )
    
    create_retrieval_grid(
        queries=text_queries[:4],
        model=wrapped_model,
        image_paths=image_paths,
        image_embeds=image_embeds,
        clip_processor=processor,
        device=device,
        images_per_query=5,
        save_path=model_output_dir / "text2img_grid.png"
    )
    
    class_labels = ['a person', 'an animal', 'a landscape']
    for idx, img_path in enumerate(image_paths[:5]):
        print(f"  Classification {idx+1}/5")
        predicted_class, confidence = zero_shot_classification(
            query_image_path=img_path,
            class_labels=class_labels,
            model=wrapped_model,
            clip_processor=processor,
            transform=transform,
            device=device,
            save_path=model_output_dir / f"classification_{idx+1}.png"
        )
        print(f"    → {predicted_class} ({confidence*100:.1f}%)")
    
    print(f"✓ Saved to: {model_output_dir}")

print("✓ Functions loaded")

In [ ]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
transform = get_clip_transform()
all_image_paths = sorted(list(val_dir.glob("*.jpg")))[:1000]
print(f"✓ Using {len(all_image_paths)} images")

In [ ]:
# LOAD & GENERATE
model = load_model_with_architecture(model_checkpoint_path, MODEL_TYPE, device)
image_embeds = precompute_image_embeddings(model, all_image_paths, transform, device)

generate_visualizations(
    model=model,
    model_name=MODEL_TYPE,
    image_paths=all_image_paths,
    image_embeds=image_embeds,
    processor=processor,
    transform=transform,
    device=device,
    output_dir=output_dir
)

del model, image_embeds
torch.cuda.empty_cache()
print("\n✅ COMPLETE!")

In [ ]:
# Download
from google.colab import files
import shutil

archive = f'/content/Lab4_Vis_{MODEL_TYPE}_FIXED'
shutil.make_archive(archive, 'zip', output_dir)
files.download(f'{archive}.zip')
print("✓ Download started!")